In [10]:
import numpy as np 
import os 
import csv 

def gf2elim(M):

    m,n = M.shape

    i=0
    j=0

    while i < m and j < n:
        # find value and index of largest element in remainder of column j
        k = np.argmax(M[i:, j]) +i

        # swap rows
        #M[[k, i]] = M[[i, k]] this doesn't work with numba
        temp = np.copy(M[k])
        M[k] = M[i]
        M[i] = temp


        aijn = M[i, j:]

        col = np.copy(M[:, j]) #make a copy otherwise M will be directly affected

        col[i] = 0 #avoid xoring pivot row with itself

        flip = np.outer(col, aijn)

        M[:, j:] = (M[:, j:] ^ flip)%2

        i += 1
        j +=1

    return M

def open_PC_matrix(csv_path):
    print(csv_path)
    with open(csv_path) as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=";")
        H = []
        for row in csv_reader:
            H.append(row)
        H = np.array(H, dtype=np.uint8)
        return H

def generate_code(csv_path):
    H_non_systematic = np.array(open_PC_matrix(csv_path),dtype=np.uint8)
    
    n = np.shape(H_non_systematic)[1]
    k = n - np.shape(H_non_systematic)[0]
    
    H_systematic = np.array(gf2elim(np.copy(H_non_systematic)),dtype=np.float32)

    redundancy_H = np.array([H_systematic[i][n-k:] for i in range(n-k)])

    H_systematic = np.concatenate((redundancy_H,np.eye(n-k)),axis=-1)
    
    G = np.concatenate((np.eye(k),redundancy_H.T),axis=-1)
    
    redundancy_G = np.array([G[i][k:] for i in range(k)])

    return(H_non_systematic.astype(np.float32), H_systematic, G, redundancy_H, redundancy_G)

def save_as_npz(csv_path, save_path="./code"):
    H_non_systematic, H_systematic, G, redundancy_H, redundancy_G = generate_code(csv_path)

    npz_path = save_path + ".npz"
    np.savez(npz_path, G=G, H_systematic=H_systematic, H_non_systematic=H_non_systematic, redundancy_H=redundancy_H, redundancy_G=redundancy_G)
    

In [ ]:
save_as_npz(csv_path="CSV/H_BCH_63_45_cycle_reduced.csv", save_path="./BCH_63_45_cycle_reduced")